In [1]:
import numpy as np

In [15]:
x1 = np.array([
        [1, 0, 0],
        [0.45, 1.24, -0.02]])

x2 = np.array([
        [0.9, -0.1, 0.3],
        [0.8, -0.6, 0.4],
        [0.2, -1.4, -0.4]])

In [22]:
def lp_dist(x1:np.ndarray, x2:np.ndarray, p:float) -> np.ndarray:
    """Get the pairwise Lp-distances between points.
    
    Args:
        x1: an m by d matrix where each row corresponds to a point to
            be compared with each point in x2.    
        x2: an n by d matrix where each row corresponds to a point to
            be compared with each point in x1. 
        p: p > 0

    Returns:
        a distance matrix of dimensions m by n where the (i,j)-th entry
        corresponds to the distance
        between points x1[i, :] and x2[j, :]
    """
    m = x1.shape[0]
    n = x2.shape[0]
    d = x1.shape[1]

    # Compare each row of x1 to each row of x2.
    # To do this, make views x1_view and x2_view
    # into x1 and x2, respectively.  x1_view and x2_view have the same
    # entries as x1 and x2, but they are repeated in a nice
    # configuration. 
    # Example:
    # x1 = np.array([
    # 	[1, 0, 0],
    # 	[0.45, 1.24, -0.02]])
    # x2 = np.array([
    # 	[0.9, -0.1, 0.3],
    # 	[0.8, -0.6, 0.4],
    # 	[0.2, -1.4, -0.4]])
    # x1_view
    # array([[[ 1.  ,  0.  ,  0.  ],
    #     [ 0.45,  1.24, -0.02]],
    #
    #    [[ 1.  ,  0.  ,  0.  ],
    #     [ 0.45,  1.24, -0.02]],
    #
    #    [[ 1.  ,  0.  ,  0.  ],
    #     [ 0.45,  1.24, -0.02]]])
    # x2_view
    # array([[[ 0.9, -0.1,  0.3],
    #         [ 0.8, -0.6,  0.4],
    #         [ 0.2, -1.4, -0.4]],
    #
    #        [[ 0.9, -0.1,  0.3],
    #         [ 0.8, -0.6,  0.4],
    #         [ 0.2, -1.4, -0.4]]])
    
    x1_view = np.broadcast_to(
        array=x1, 
        shape=(n, m, d)
    )
    
    x2_view = np.broadcast_to(
        array=x2, 
        shape=(m, n, d)
    )

    # Make sure our views have the same dimensions.
    x2_view_2 = np.swapaxes(x2_view, axis1=0, axis2=1)
    # Example:
    # x2_view_2
    # array([[[ 0.9, -0.1,  0.3],
    #         [ 0.9, -0.1,  0.3]],
    #
    #        [[ 0.8, -0.6,  0.4],
    #         [ 0.8, -0.6,  0.4]],
    #
    #        [[ 0.2, -1.4, -0.4],
    #         [ 0.2, -1.4, -0.4]]])
    
    # Now, x1_view and x2_view_2 are broadcast-able.

    return np.power(
        np.sum(
            np.power(
                np.abs(x1_view - x2_view_2), 
                p
            ), 
            axis=2
        ).T, 
        1.0/p
    )

In [17]:
x1

array([[ 1.  ,  0.  ,  0.  ],
       [ 0.45,  1.24, -0.02]])

In [18]:
x2

array([[ 0.9, -0.1,  0.3],
       [ 0.8, -0.6,  0.4],
       [ 0.2, -1.4, -0.4]])

In [23]:
lp_dist(x1, x2, 2)

array([[0.33166248, 0.74833148, 1.66132477],
       [1.44931018, 1.91950514, 2.67889903]])